In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
import seaborn as sns
%matplotlib inline

In [3]:
MONTHS_GEN = {
    1: "января",
    2: "февраля",
    3: "марта",
    4: "апреля",
    5: "мая",
    6: "июня",
    7: "июля",
    8: "августа",
    9: "сентября",
    10: "октября",
    11: "ноября",
    12: "декабря"
}

In [6]:
data = pd.read_csv('weather_data.csv', sep=',').drop(['text'], axis='columns')

In [8]:
data.head()

,21-22,8-10,8-9,23-27,16-20,26-29,08-09,9,20-25,10-12,...,14-16,15-17,3-4,26-28,22-24,25,12-14,month,day_start,day_end
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,8,9,9
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,10,10
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,8,9,9
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,8,8
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,8,9,9


In [11]:
month_list = list(MONTHS_GEN.values())
month_list.append('month')

['января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря', 'month']


In [13]:
m_data = data.loc[:, month_list]

In [14]:
m_data.head()

,января,февраля,марта,апреля,мая,июня,июля,августа,сентября,октября,ноября,декабря,month
0,0,0,0,0,0,0,0,1,0,0,0,0,8
1,0,0,0,0,0,0,0,1,0,0,0,0,8
2,0,0,0,0,0,0,0,1,0,0,0,0,8
3,0,0,0,0,0,0,0,1,0,0,0,0,8
4,0,0,0,0,0,0,0,1,0,0,0,0,8


In [15]:
m_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11875 entries, 0 to 11874
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   января    11875 non-null  int64
 1   февраля   11875 non-null  int64
 2   марта     11875 non-null  int64
 3   апреля    11875 non-null  int64
 4   мая       11875 non-null  int64
 5   июня      11875 non-null  int64
 6   июля      11875 non-null  int64
 7   августа   11875 non-null  int64
 8   сентября  11875 non-null  int64
 9   октября   11875 non-null  int64
 10  ноября    11875 non-null  int64
 11  декабря   11875 non-null  int64
 12  month     11875 non-null  int64
dtypes: int64(13)
memory usage: 1.2 MB


In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [20]:
m_x= m_data.drop('month', axis=1)
m_y = m_data['month'].astype('int')

In [23]:
m_x.shape, m_y.shape

((11875, 12), (11875,))

In [25]:
m_x_train, m_x_valid, m_y_train, m_y_valid = train_test_split(m_x, m_y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [26]:
m_x_train.shape, m_x_valid.shape

((8312, 12), (3563, 12))

In [27]:
first_tree = DecisionTreeClassifier(random_state=17)

In [33]:
np.mean(cross_val_score(first_tree, m_x_train, m_y_train, cv=5))

0.9977141046041365

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, m_x_train, m_y_train, cv=5))

0.9959093398979559

In [35]:
from sklearn.model_selection import GridSearchCV

In [40]:
tree_params = {'max_depth': np.arange(1, 15), 'max_features':[.5, .7, 1]}

In [41]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [43]:
%%time
tree_grid.fit(m_x_train, m_y_train);

Wall time: 526 ms


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'max_features': [0.5, 0.7, 1]})

In [44]:
tree_grid.best_score_, tree_grid.best_params_


(0.9979547784910195, {'max_depth': 12, 'max_features': 0.5})

In [51]:
tree_valid_pred = tree_grid.predict(m_x_valid)

In [52]:
from sklearn.metrics import accuracy_score

In [53]:
accuracy_score(m_y_valid, tree_valid_pred)

0.9994386752736458

In [54]:
from sklearn.metrics import confusion_matrix

In [55]:
confusion_matrix(m_y_valid, tree_valid_pred)

array([[283,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 276,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 265,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 311,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 305,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 344,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 350,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 296,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 296,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 309,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 271]],
      dtype=int64)

In [56]:
from sklearn.tree import export_graphviz

In [60]:
second_tree = DecisionTreeClassifier(max_depth=12).fit(m_x_train, m_y_train)
second_tree.score(m_x_valid, m_y_valid)


0.9991580129104687